<a href="https://colab.research.google.com/github/Abinaya-512/LangGraph/blob/main/Copy_of_Untitled50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langgraph langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling la

In [ ]:
from typing import TypedDict, Literal, Dict, Any, Callable, Optional, List, Tuple
import inspect
import sys

# -----------------------------
# 0) Try to import langgraph; if unavailable, use a Mock implementation
# -----------------------------
try:
    from langgraph.graph import StateGraph
    from langgraph.checkpoint.memory import MemorySaver
    LANGGRAPH_AVAILABLE = True
except Exception as e:
    LANGGRAPH_AVAILABLE = False

    # Minimal Mock classes to emulate the behavior needed for the tutorial.
    class MemorySaver:
        def __init__(self):
            self.storage = []
        def save(self, state: Dict[str, Any]):
            self.storage.append(dict(state))
        def list(self):
            return list(self.storage)

    class CompiledMockGraph:
        def __init__(self, nodes: Dict[str, Callable], edges: Dict[str, Any], entry: str, finish: str, checkpointer: Optional[MemorySaver]):
            self.nodes = nodes
            self.edges = edges
            self.entry = entry
            self.finish = finish
            self.checkpointer = checkpointer

        def invoke(self, state: Dict[str, Any]) -> Dict[str, Any]:
            # simple synchronous traversal
            current = self.entry
            visited = []
            while True:
                visited.append(current)
                node_fn = self.nodes[current]
                state = node_fn(state)
                # checkpoint
                if self.checkpointer is not None:
                    self.checkpointer.save(state)
                # finish condition
                if current == self.finish:
                    break
                # determine next
                transition = self.edges.get(current)
                if transition is None:
                    raise RuntimeError(f"No transition defined from node: {current}")
                # transition may be a string (deterministic), or callable router
                if isinstance(transition, str):
                    next_node = transition
                elif callable(transition):
                    next_node = transition(state)
                else:
                    raise RuntimeError(f"Unknown transition type from {current}: {transition}")
                if next_node not in self.nodes:
                    raise ValueError(f"Found edge ending at unknown node `{next_node}`")
                current = next_node
            return state

    class StateGraph:
        def __init__(self, schema: Any = None):
            self._nodes: Dict[str, Callable] = {}
            # edges maps start_node -> either end_node_name or routing_fn
            self._edges: Dict[str, Any] = {}
            self._entry: Optional[str] = None
            self._finish: Optional[str] = None

        def add_node(self, name: str, fn: Callable):
            self._nodes[name] = fn

        def add_edge(self, start: str, end: str):
            # store deterministic edge
            self._edges[start] = end

        def add_conditional_edges(self, start: str, router: Callable[[Dict[str, Any]], Any]):
            self._edges[start] = router

        def set_entry_point(self, name: str):
            self._entry = name

        def set_finish_point(self, name: str):
            self._finish = name

        # compile returns an object with invoke(state)
        def compile(self, checkpointer: Optional[MemorySaver] = None, **kwargs):
            # validate
            all_targets = set()
            for v in self._edges.values():
                if isinstance(v, str):
                    all_targets.add(v)
                elif callable(v):
                    # we cannot statically know the return set; skip
                    pass
            for target in all_targets:
                if target not in self._nodes:
                    raise ValueError(f"Found edge ending at unknown node `{target}`")
            if self._entry is None or self._finish is None:
                raise RuntimeError("Entry and finish points must be set")
            return CompiledMockGraph(self._nodes, self._edges, self._entry, self._finish, checkpointer)

# -----------------------------
# 1) Define the State schema
# -----------------------------
class MyState(TypedDict, total=False):
    question: str
    docs: str
    answer: str
    use_retriever: bool

# -----------------------------
# 2) Define nodes
# -----------------------------

def classifier_node(state: MyState) -> MyState:
    q = state.get("question", "").lower()
    keywords = ["who", "when", "where", "facts", "what is", "define"]
    state["use_retriever"] = any(k in q for k in keywords)
    return state


def retriever_node(state: MyState) -> MyState:
    question = state.get("question", "")
    state["docs"] = f"[Retrieved context for: {question}] Relevant facts..."
    return state


def generator_node(state: MyState) -> MyState:
    q = state.get("question", "")
    docs = state.get("docs")
    if docs:
        state["answer"] = f"Answer (with docs): Based on context: {docs} --> Answer to '{q}'"
    else:
        state["answer"] = f"Answer (no docs): A short answer to '{q}'"
    return state

# -----------------------------
# 3) Assemble StateGraph
# -----------------------------
workflow = StateGraph(MyState)
workflow.add_node("classifier", classifier_node)
workflow.add_node("retriever", retriever_node)
workflow.add_node("generator", generator_node)
workflow.set_entry_point("classifier")

# -----------------------------
# 4) Conditional routing
# -----------------------------
from typing import cast

def route_after_classify(state: MyState) -> Literal["retriever", "generator"]:
    return "retriever" if state.get("use_retriever") else "generator"

workflow.add_conditional_edges("classifier", route_after_classify)
workflow.add_edge("retriever", "generator")
workflow.set_finish_point("generator")

# -----------------------------
# 5) Diagnostics before compiling
# -----------------------------
print("--- DIAGNOSTICS: nodes registered in workflow ---")
try:
    nodes = getattr(workflow, "nodes", None) or getattr(workflow, "_nodes", None)
    if isinstance(nodes, dict):
        print("nodes:", list(nodes.keys()))
    else:
        print("nodes object:", nodes)
except Exception as e:
    print("could not read workflow.nodes:", e)

print("--- DIAGNOSTICS: edges registered in workflow ---")
try:
    edges = (
        getattr(workflow, "edges", None)
        or getattr(workflow, "_edges", None)
        or getattr(workflow, "_transitions", None)
        or getattr(workflow, "_edges", None)
    )
    print("edges:", edges)
except Exception as e:
    print("could not read workflow.edges:", e)

print("If any of the printed 'edges' show a <function, remove the bad add_edge call from your environment or restart the kernel.")

# -----------------------------
# 6) Compile with MemorySaver
# -----------------------------
memory = MemorySaver()
try:
    graph = workflow.compile(checkpointer=memory)
    print("Compiled graph successfully.")
except Exception as exc:
    print("Compile failed with error:", exc)
    raise

# -----------------------------
# 7) Run examples and tests
# -----------------------------

def run_example(question: str, thread_id: str) -> Dict[str, Any]:
    initial_state: MyState = {"question": question}
    config = {"configurable": {"thread_id": thread_id}}
    final_state = graph.invoke(initial_state, config=config)
    print("QUESTION:", question)
    print("USE_RETRIEVER:", final_state.get("use_retriever"))
    print("DOCS:", final_state.get("docs"))
    print("ANSWER:", final_state.get("answer"))
    return final_state

# Minimal tests (assertions) to ensure behavior is consistent.
# Add more tests here if you want to exercise other branches.

def _run_tests():
    print("--- Running quick tests ---")
    s1 = run_example("Who invented the telephone?", thread_id="test_thread_1")
    assert s1.get("use_retriever") is True, "Expected retriever to be used for factual question"
    assert "Retrieved context for" in (s1.get("docs") or ""), "Expected docs to be populated"

    s2 = run_example("Give a short analogy for recursion.", thread_id="test_thread_2")
    assert s2.get("use_retriever") is False, "Expected retriever NOT to be used for opinion question"
    assert s2.get("docs") is None or s2.get("docs") == "", "Expected no docs for short opinion question"

    s3 = run_example("What is the capital of France?", thread_id="test_thread_3")
    assert s3.get("use_retriever") is True

    print("All tests passed.")

if __name__ == "__main__":
    print(f"LANGGRAPH_AVAILABLE={LANGGRAPH_AVAILABLE}")
    _run_tests()

# End of file


--- DIAGNOSTICS: nodes registered in workflow ---
nodes: ['classifier', 'retriever', 'generator']
--- DIAGNOSTICS: edges registered in workflow ---
edges: {('generator', '__end__'), ('__start__', 'classifier'), ('retriever', 'generator')}
If any of the printed 'edges' show a <function, remove the bad add_edge call from your environment or restart the kernel.
Compiled graph successfully.
LANGGRAPH_AVAILABLE=True
--- Running quick tests ---
QUESTION: Who invented the telephone?
USE_RETRIEVER: True
DOCS: [Retrieved context for: Who invented the telephone?] Relevant facts...
ANSWER: Answer (with docs): Based on context: [Retrieved context for: Who invented the telephone?] Relevant facts... --> Answer to 'Who invented the telephone?'
QUESTION: Give a short analogy for recursion.
USE_RETRIEVER: False
DOCS: None
ANSWER: Answer (no docs): A short answer to 'Give a short analogy for recursion.'
QUESTION: What is the capital of France?
USE_RETRIEVER: True
DOCS: [Retrieved context for: What is th